In [148]:
import torch
import torch.nn as nn
from torch.utils import data
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import pandas as pd
# import keras
import sklearn
import pickle
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Variable

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from torchvision.models import resnet18

In [149]:
# 数据集的相对地址
dataset_train_dir = './EMNIST_Byclass_Small/emnist_train.pkl'
dataset_test_dir = './EMNIST_Byclass_Small/emnist_test.pkl'

# 将数据集文件解压缩，读取为字典（按照dataset_description.txt文件所示）
dataset_train_dict = pickle.load(file=open(dataset_train_dir, 'rb'))
dataset_test_dict = pickle.load(file=open(dataset_test_dir, 'rb'))

num_epochs = 10
batch_size = 128
learning_rate = 0.5

class EMNIST(data.Dataset):
    def __init__(self,  X, y, transform=None):
        self.transform = transform
        self.X = X
        self.y = y
        
    def __len__ (self):
        return len(self.X)
    
    def __getitem__(self, index):
        self.XX = self.X[index]
        self.yy = self.y[index]
        if self.transform:
            self.XX = self.transform(self.XX)
        return self.XX, self.yy

x_train = dataset_train_dict['data'].astype('float32')
y_train = dataset_train_dict['labels'].reshape(-1).astype('int64')
#y_train = nn.functional.one_hot(torch.from_numpy(y_train))
y_train = torch.tensor(y_train).to(torch.int64)
# y_train = np.asarray(y_train, dtype=float)

x_test = dataset_test_dict['data'].astype('float32')
y_test = dataset_test_dict['labels'].reshape(-1).astype('int64')
#y_test = nn.functional.one_hot(torch.from_numpy(y_test))
y_test = torch.tensor(y_test).to(torch.int64)
# y_test = np.asarray(y_test, dtype=float)

emnist_train = EMNIST(x_train, y_train, transform=T.Compose([T.ToTensor()]))
train_iter = data.DataLoader(emnist_train, batch_size=batch_size, shuffle=True)

emnist_test = EMNIST(x_test, y_test, transform=T.Compose([T.ToTensor()]))
test_iter = data.DataLoader(emnist_test, batch_size=batch_size, shuffle=False)


In [150]:
net = nn.Sequential(
    nn.Flatten(),
    
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(256,256),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(256,62)
    
)

In [151]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import numpy as np
import os
import pandas as pd
import torchvision.transforms as transforms
from d2l import torch as d2l
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

class Accumulator(object):
    """累加器"""
    def __init__(self, n):
        # 创建len=n的list
        self.data = [0.0]*n
        
    def add(self, *args):
        # data累加args
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        # 重置累加器
        self.data = [0.0]*len(self.data)
        
    def __getitem__(self, idx):
        return self.data[idx]

class Timer(object):
    """计时器"""
    def __init__(self):
        self.times=[]
        self.start()
        
    def start(self):
        # 记录开始时间戳
        self.tik = time.time()
    
    def stop(self):
        self.times.append(time.time() - self.tik)
        return self.times[-1]
    
    def avg(self):
        return sum(self.times) / len(self.times)
    
    def sum(self):
        return sum(self.times)
    
    def cumsum(self):
        return np.array(self.times).cumsum().tolist()    

argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
size = lambda x, *args, **kwargs: x.numel(*args, **kwargs)

def cal_correct(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)

    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))

def evaluate_accuracy(net, data_iter, device=None):
    """计算模型在数据集上的准确率"""
    if isinstance(net, torch.nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device
    
    metric = Accumulator(2)
    
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(cal_correct(net(X), y), size(y))
    return metric[0] / metric[1]

###########################################################################################################
# FUNCTION: trainer()
###########################################################################################################
def trainer(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == torch.nn.Linear or type(m) == torch.nn.Conv2d:
            torch.nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    
    device = torch.device(device)
    print('training on', device)
    net.to(device)
    
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss_fuction = torch.nn.CrossEntropyLoss()
    
    timer = Timer()
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            # y_hat = np.argmax(y_hat.detach().numpy(), axis=1)
            loss = loss_fuction(y_hat, y)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                metric.add(loss*X.shape[0], cal_correct(y_hat, y),X.shape[0])
            timer.stop()
            
        train_loss = metric[0] / metric[2]
        train_acc = metric[1] / metric[2]
        test_acc = evaluate_accuracy(net, test_iter)
        print(f'epoch{epoch+1}:')
        print(f'loss{train_loss:.3f}, train acc{train_acc:.3f}, '
            f'test acc {test_acc:.3f}')
        print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
            f'on {str(device)}')

In [152]:
trainer(net, train_iter, test_iter, num_epochs, 0.0005, torch.device('cuda:0'))

training on cuda:0
epoch1:
loss8.103, train acc0.146, test acc 0.269
374045.5 examples/sec on cuda:0
epoch2:
loss3.187, train acc0.243, test acc 0.329
182844.9 examples/sec on cuda:0
epoch3:
loss2.858, train acc0.300, test acc 0.376
119529.9 examples/sec on cuda:0
epoch4:
loss2.589, train acc0.345, test acc 0.436
90546.0 examples/sec on cuda:0
epoch5:
loss2.389, train acc0.397, test acc 0.482
72176.7 examples/sec on cuda:0
epoch6:
loss2.181, train acc0.435, test acc 0.518
59964.6 examples/sec on cuda:0
epoch7:
loss2.038, train acc0.463, test acc 0.551
51425.6 examples/sec on cuda:0
epoch8:
loss1.928, train acc0.487, test acc 0.579
44955.7 examples/sec on cuda:0
epoch9:
loss1.852, train acc0.504, test acc 0.604
40006.3 examples/sec on cuda:0
epoch10:
loss1.761, train acc0.523, test acc 0.626
36788.2 examples/sec on cuda:0
